In [1]:
import pandas as pd
from paquo.projects import QuPathProject

from pathlib import Path
from paquo.projects import QuPathProject
from paquo.images import QuPathImageType
from paquo.classes import QuPathPathClass
from shapely.geometry import Point, Polygon, MultiPoint
import numpy as np
import matplotlib.pyplot as plt
from distutils.dir_util import copy_tree
import glob


08:42:11.852 [main] [WARN ] q.l.objects.classes.PathClassFactory - PathClassFactory is deprecated since v0.4.0 and will be removed - use PathClass methods instead


/tmp/ipykernel_30455/4224394371.py:11: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree


In [2]:
def get_qupath_project_by_image_name(dataset, image_name : str, root_path=None) ->str:
    
    project_name =  dataset[dataset['Image Name'].str.contains(image_name)]['Project ID'].iloc[0]
    project_name = 'ProjectQuPath_' + project_name[project_name.find('_')+1:]
    if root_path is not None:
        nissl_id = project_name[project_name.find('Nissl'):]
        project_name = root_path + nissl_id + '/' + project_name
        dir_path = project_name + '/*.qpproj'
        #print(f'dir_path {dir_path}')
        project_name = glob.glob(dir_path)
        #print(f'project_name {project_name}')
        assert len(project_name) == 1
        project_name = project_name[0]
    return project_name

In [3]:
dataset_filepath = '../20231102/datasets/ALL_ProjectQuPath_Metadata information.csv'
qupath_project_root_path = '/media/LNMC/LBR_Analysis/WithPredictions/'

dataset = pd.read_csv(dataset_filepath)

pixel_size = 0.3460130331522824

layers_color = {"Layer 1": "#ff0000"
                , "Layer 2":"#0f00f0"
                , "Layer 3":"#0000ff"
                , "Layer 2/3":"#80801a"
                , "Layer 4":"#003333"
                , "Layer 5":"#00f5ff"
                , "Layer 6 a":"#663366"
                , "Layer 6 b":"#00ff00"
               }

FileNotFoundError: [Errno 2] No such file or directory: '../20231102/datasets/ALL_ProjectQuPath_Metadata information.csv'

In [ ]:
#ml_result = pd.read_csv('../../ML_Layer_prediction/test_features_with_prediction.csv')
ml_result = pd.read_csv('/gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/Machine-Learning/Predictions/global_predictions.csv',  low_memory=False)

In [5]:
image_names = np.unique(ml_result['Image'])

In [22]:
from collections import defaultdict
image_per_projects = defaultdict(list)
for image_name in image_names:
    qupath_project = get_qupath_project_by_image_name(dataset, image_name, qupath_project_root_path)
    image_per_projects[qupath_project].append(image_name)


In [33]:
done = []
nb_images = len(image_names)
#for image_name in image_names:
#    print(f' search {image_name} ')

#    qupath_project = get_qupath_project_by_image_name(dataset, image_name, qupath_project_root_path)

#    print(f'Image name {image_name}, qupath {qupath_project}')

for qupath_project in image_per_projects.keys():
    
    print(f'Open project {qupath_project}')
    with QuPathProject(qupath_project, mode='r+') as qp:
        print(f'process {image_per_projects[qupath_project]}')
        for image_name in image_per_projects[qupath_project]:

            image_ml_result = ml_result[ml_result['Image']==image_name]

            annotations = []
            for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 2/3",  "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
            #for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
                layer_points = image_ml_result[image_ml_result['RF_prediction'] == layer][['Centroid X µm', 'Centroid Y µm']].to_numpy()
                #print(f'Layer points {layer_points}')
                layer_points = layer_points / pixel_size
                if layer_points.size > 0:
                    annotations.append({ layer: MultiPoint(layer_points)})

            assert len(annotations) > 0
            #print("readed", qp.name)  
            #print(f'---INFO: there are {len(qp.images)}  images in this project')
            found = False
            for image in qp.images:
                if image.image_name.find(image_name) > -1:
                    for index, annotation in enumerate(annotations):
                        for name, roi in annotation.items():
                            path_class = QuPathPathClass(name = name + '_prediction', color=layers_color[name])

                            annotation = image.hierarchy.add_annotation(roi=roi, path_class=path_class)
                    found = True
                    done.append(image_name)
                if found: 
                    break
        print(f'Close project')
    
    print(f'project closed')        

    print('INFO: Done')


    

Open project /media/LNMC/LBR_Analysis/WithPredictions/Nissl_1/ProjectQuPath_01413827_RH_Nissl_1/ProjectQuPath_01413827_RH_Nissl_1.qpproj
process ['SLD_0000400.vsi - 20x_02', 'SLD_0000401.vsi - 20x_04', 'SLD_0000402.vsi - 20x_01', 'SLD_0000402.vsi - 20x_03', 'SLD_0000402.vsi - 20x_04', 'SLD_0000403.vsi - 20x_01', 'SLD_0000404.vsi - 20x_01', 'SLD_0000404.vsi - 20x_03', 'SLD_0000405.vsi - 20x_01', 'SLD_0000405.vsi - 20x_03', 'SLD_0000406.vsi - 20x_02', 'SLD_0000408.vsi - 20x_01', 'SLD_0000409.vsi - 20x_01', 'SLD_0000412.vsi - 20x_02', 'SLD_0000413.vsi - 20x_02']
Close project
15:21:07.421 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_1/ProjectQuPath_01413827_RH_Nissl_1/ProjectQuPath_01413827_RH_Nissl_1.qpproj.tmp
15:21:07.455 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_1/ProjectQuPath_01413827_RH_Nissl_1/ProjectQuPath_01413827_RH_Nis

Saving skipped for 'SLD_0000463.vsi - 20x_01': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000463.vsi' not reachable
Saving skipped for 'SLD_0000463.vsi - 20x_02': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000463.vsi' not reachable
Saving skipped for 'SLD_0000463.vsi - 20x_03': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000463.vsi' not reachable
Saving skipped for 'SLD_0000463.vsi - 20x_04': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000463.vsi' not reachable
Saving skipped for 'SLD_0000463.vsi - 20x_05': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000463.vsi' not reachable
Saving skipped for 'SLD_0000464.vsi - 20x_01': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000464.vsi' not reachable
Saving skipped for 'SLD_0000464.vsi - 20x_02': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000464.vsi'

Saving skipped for 'SLD_0000445.vsi - 20x_07': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000445.vsi' not reachable
Saving skipped for 'SLD_0000445.vsi - 20x_08': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000445.vsi' not reachable
Saving skipped for 'SLD_0000446.vsi - 20x_01': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000446.vsi' not reachable
Saving skipped for 'SLD_0000446.vsi - 20x_02': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000446.vsi' not reachable
Saving skipped for 'SLD_0000446.vsi - 20x_03': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000446.vsi' not reachable
Saving skipped for 'SLD_0000446.vsi - 20x_04': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000446.vsi' not reachable
Saving skipped for 'SLD_0000446.vsi - 20x_05': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000446.vsi'

Saving skipped for 'SLD_0000456.vsi - 20x_01': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000456.vsi' not reachable
Saving skipped for 'SLD_0000456.vsi - 20x_02': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000456.vsi' not reachable
Saving skipped for 'SLD_0000456.vsi - 20x_03': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000456.vsi' not reachable
Saving skipped for 'SLD_0000456.vsi - 20x_04': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000456.vsi' not reachable
Saving skipped for 'SLD_0000457.vsi - 20x_01': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000457.vsi' not reachable
Saving skipped for 'SLD_0000457.vsi - 20x_02': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000457.vsi' not reachable
Saving skipped for 'SLD_0000457.vsi - 20x_03': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000457.vsi'

Close project
15:26:44.517 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413828_RH_Nissl_2/ProjectQuPath_01413828_RH_Nissl_2.qpproj.tmp
15:26:44.551 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413828_RH_Nissl_2/ProjectQuPath_01413828_RH_Nissl_2.qpproj.backup
15:26:44.577 [main] [DEBUG] qupath.lib.projects.DefaultProject - Renaming project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413828_RH_Nissl_2/ProjectQuPath_01413828_RH_Nissl_2.qpproj
project closed
INFO: Done
Open project /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413829_RH_Nissl_2/ProjectQuPath_01413829_RH_Nissl_2.qpproj
process ['SLD_0000479.vsi - 20x_06', 'SLD_0000480.vsi - 20x_01', 'SLD_0000480.vsi - 20x_02', 'SLD_0000480.vsi - 20x_03', 'SLD_0000480.vsi - 20x_04', 'SLD_0000480.vsi - 20x_05'

...

Saving skipped for 'SLD_0000473.vsi - 20x_01': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000473.vsi' not reachable
Saving skipped for 'SLD_0000473.vsi - 20x_02': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000473.vsi' not reachable
Saving skipped for 'SLD_0000473.vsi - 20x_03': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000473.vsi' not reachable
Saving skipped for 'SLD_0000473.vsi - 20x_04': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000473.vsi' not reachable
Saving skipped for 'SLD_0000473.vsi - 20x_05': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000473.vsi' not reachable
Saving skipped for 'SLD_0000473.vsi - 20x_06': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000473.vsi' not reachable
Saving skipped for 'SLD_0000474.vsi - 20x_01': uri 'file:///Y:/2021_LayerBoundariesRefinement/LBR_RAWData/Nissl_2/SLD_0000474.vsi'

Close project
15:27:16.323 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413829_RH_Nissl_2/ProjectQuPath_01413829_RH_Nissl_2.qpproj.tmp
15:27:16.356 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413829_RH_Nissl_2/ProjectQuPath_01413829_RH_Nissl_2.qpproj.backup
15:27:16.380 [main] [DEBUG] qupath.lib.projects.DefaultProject - Renaming project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413829_RH_Nissl_2/ProjectQuPath_01413829_RH_Nissl_2.qpproj
project closed
INFO: Done
Open project /media/LNMC/LBR_Analysis/WithPredictions/Nissl_3/ProjectQuPath_1413828_LH_Nissl_3/ProjectQuPath_1413828_LH_Nissl_3.qpproj
process ['SLD_0000506.vsi - 20x_04', 'SLD_0000506.vsi - 20x_05', 'SLD_0000506.vsi - 20x_06', 'SLD_0000507.vsi - 20x_01', 'SLD_0000507.vsi - 20x_04', 'SLD_0000508.vsi - 20x_01', 

Close project
15:37:50.988 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_4/ProjectQuPath_1443459_RH_Nissl_4/ProjectQuPath_1443459_RH_Nissl_4.qpproj.tmp
15:37:51.034 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_4/ProjectQuPath_1443459_RH_Nissl_4/ProjectQuPath_1443459_RH_Nissl_4.qpproj.backup
15:37:51.058 [main] [DEBUG] qupath.lib.projects.DefaultProject - Renaming project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_4/ProjectQuPath_1443459_RH_Nissl_4/ProjectQuPath_1443459_RH_Nissl_4.qpproj
project closed
INFO: Done
Open project /media/LNMC/LBR_Analysis/WithPredictions/Nissl_4/ProjectQuPath_1443460_RH_Nissl_4/ProjectQuPath_1443460_RH_Nissl_4.qpproj
process ['SLD_0000560.vsi - 20x_01', 'SLD_0000560.vsi - 20x_02', 'SLD_0000560.vsi - 20x_03', 'SLD_0000560.vsi - 20x_04', 'SLD_0000560.vsi - 20x_05', 'SLD_0000560.vsi - 20x_06', 'SLD_0

Close project
15:42:44.958 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_4/ProjectQuPath_1443460_RH_Nissl_4/ProjectQuPath_1443460_RH_Nissl_4.qpproj.tmp
15:42:45.002 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_4/ProjectQuPath_1443460_RH_Nissl_4/ProjectQuPath_1443460_RH_Nissl_4.qpproj.backup
15:42:45.025 [main] [DEBUG] qupath.lib.projects.DefaultProject - Renaming project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_4/ProjectQuPath_1443460_RH_Nissl_4/ProjectQuPath_1443460_RH_Nissl_4.qpproj
project closed
INFO: Done
Open project /media/LNMC/LBR_Analysis/WithPredictions/Nissl_5/ProjectQuPath_1443459_LH_Nissl_5/ProjectQuPath_1443459_LH_Nissl_5.qpproj
process ['SLD_0000582.vsi - 20x_02', 'SLD_0000582.vsi - 20x_04', 'SLD_0000582.vsi - 20x_05', 'SLD_0000582.vsi - 20x_06', 'SLD_0000583.vsi - 20x_01', 'SLD_0000583.vsi - 20x_02', 'SLD_0

Close project
16:10:55.537 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_6/ProjectQuPath_1449922_RH_Nissl_6/ProjectQuPath_1449922_RH_Nissl_6.qpproj.tmp
16:10:55.588 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_6/ProjectQuPath_1449922_RH_Nissl_6/ProjectQuPath_1449922_RH_Nissl_6.qpproj.backup
16:10:55.613 [main] [DEBUG] qupath.lib.projects.DefaultProject - Renaming project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_6/ProjectQuPath_1449922_RH_Nissl_6/ProjectQuPath_1449922_RH_Nissl_6.qpproj
project closed
INFO: Done
Open project /media/LNMC/LBR_Analysis/WithPredictions/Nissl_6/ProjectQuPath_1449922_LH_Nissl_6/ProjectQuPath_1449922_LH_Nissl_6.qpproj
process ['SLD_0000748.vsi - 20x_04', 'SLD_0000748.vsi - 20x_05', 'SLD_0000748.vsi - 20x_06', 'SLD_0000749.vsi - 20x_01', 'SLD_0000749.vsi - 20x_02', 'SLD_0000749.vsi - 20x_03', 'SLD_0

In [43]:
done = []
nb_images = len(image_names)
#for image_name in image_names:
#    print(f' search {image_name} ')

#    qupath_project = get_qupath_project_by_image_name(dataset, image_name, qupath_project_root_path)

#    print(f'Image name {image_name}, qupath {qupath_project}')

for qupath_project in image_per_projects.keys():
    if qupath_project.find('ProjectQuPath_01413829_RH_Nissl_2') == -1:
        
        continue
    print(f'Open project {qupath_project}')
    with QuPathProject(qupath_project, mode='r+') as qp:
        print(f'process {image_per_projects[qupath_project]}')
        for image_name in image_per_projects[qupath_project]:

            image_ml_result = ml_result[ml_result['Image']==image_name]

            annotations = []
            for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 2/3",  "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
            #for index, layer in enumerate(["Layer 1", "Layer 2", "Layer 3", "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]):
                layer_points = image_ml_result[image_ml_result['RF_prediction'] == layer][['Centroid X µm', 'Centroid Y µm']].to_numpy()
                #print(f'Layer points {layer_points}')
                layer_points = layer_points / pixel_size
                if layer_points.size > 0:
                    annotations.append({ layer: MultiPoint(layer_points)})

            assert len(annotations) > 0
            #print("readed", qp.name)  
            #print(f'---INFO: there are {len(qp.images)}  images in this project')
            found = False
            for image in qp.images:
                if image.image_name.find(image_name) > -1:
                    for index, annotation in enumerate(annotations):
                        for name, roi in annotation.items():
                            path_class = QuPathPathClass(name = name + '_prediction', color=layers_color[name])

                            annotation = image.hierarchy.add_annotation(roi=roi, path_class=path_class)
                    found = True
                    done.append(image_name)
                if found: 
                    break
        print(f'Close project')
    
    print(f'project closed')        

    print('INFO: Done')


    

Open project /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413829_RH_Nissl_2/ProjectQuPath_01413829_RH_Nissl_2.qpproj
process ['SLD_0000479.vsi - 20x_06', 'SLD_0000480.vsi - 20x_01', 'SLD_0000480.vsi - 20x_02', 'SLD_0000480.vsi - 20x_03', 'SLD_0000480.vsi - 20x_04', 'SLD_0000480.vsi - 20x_05', 'SLD_0000481.vsi - 20x_01', 'SLD_0000481.vsi - 20x_02', 'SLD_0000481.vsi - 20x_03', 'SLD_0000481.vsi - 20x_04', 'SLD_0000482.vsi - 20x_02', 'SLD_0000482.vsi - 20x_04', 'SLD_0000483.vsi - 20x_03', 'SLD_0000483.vsi - 20x_04', 'SLD_0000484.vsi - 20x_01', 'SLD_0000484.vsi - 20x_02', 'SLD_0000484.vsi - 20x_03', 'SLD_0000485.vsi - 20x_03', 'SLD_0000486.vsi - 20x_01', 'SLD_0000486.vsi - 20x_02', 'SLD_0000488.vsi - 20x_02', 'SLD_0000489.vsi - 20x_01', 'SLD_0000489.vsi - 20x_02', 'SLD_0000490.vsi - 20x_01', 'SLD_0000491.vsi - 20x_01', 'SLD_0000493.vsi - 20x_03', 'SLD_0000493.vsi - 20x_06']
16:42:58.149 [Finalizer] [DEBUG] q.l.i.s.b.BioFormatsImageServer$ReaderPool - Cleaner 1 called for

16:43:10.706 [Finalizer] [DEBUG] q.l.i.s.b.BioFormatsImageServer$ReaderPool - Cleaner 1 called for loci.formats.ImageReader@5a19085a (/gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/LBR_RAWData/Nissl_2/SLD_0000446.vsi)
16:43:10.706 [Finalizer] [DEBUG] loci.common.Location - Location.mapFile: embedded-stream.raw -> null
16:43:10.706 [Finalizer] [DEBUG] loci.common.Location - Location.mapFile: embedded-stream.raw -> null
16:43:10.706 [Finalizer] [DEBUG] q.l.i.s.b.BioFormatsImageServer$ReaderPool - Cleaner 1 called for loci.formats.ImageReader@3693f393 (/gpfs/bbp.cscs.ch/project/proj53/LayerBoundariesProject/LBR_RAWData/Nissl_2/SLD_0000446.vsi)
16:43:10.706 [Finalizer] [DEBUG] loci.common.Location - Location.mapFile: embedded-stream.raw -> null
16:43:10.706 [Finalizer] [DEBUG] loci.common.Location - Location.mapFile: embedded-stream.raw -> null
16:43:10.706 [Finalizer] [DEBUG] q.l.i.s.b.BioFormatsImageServer$ReaderPool - Cleaner 1 called for loci.formats.ImageReader@547b1e64 (/gpf

Close project
16:48:26.560 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413829_RH_Nissl_2/ProjectQuPath_01413829_RH_Nissl_2.qpproj.tmp
16:48:26.621 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413829_RH_Nissl_2/ProjectQuPath_01413829_RH_Nissl_2.qpproj.backup
16:48:26.660 [main] [DEBUG] qupath.lib.projects.DefaultProject - Renaming project to /media/LNMC/LBR_Analysis/WithPredictions/Nissl_2/ProjectQuPath_01413829_RH_Nissl_2/ProjectQuPath_01413829_RH_Nissl_2.qpproj
project closed
INFO: Done


In [44]:
len(first_done)

348

In [45]:
first_done.extend(done)

In [46]:
len(first_done)

375

In [47]:
first_done

['SLD_0000400.vsi - 20x_02',
 'SLD_0000401.vsi - 20x_04',
 'SLD_0000402.vsi - 20x_01',
 'SLD_0000402.vsi - 20x_03',
 'SLD_0000402.vsi - 20x_04',
 'SLD_0000403.vsi - 20x_01',
 'SLD_0000404.vsi - 20x_01',
 'SLD_0000404.vsi - 20x_03',
 'SLD_0000405.vsi - 20x_01',
 'SLD_0000405.vsi - 20x_03',
 'SLD_0000406.vsi - 20x_02',
 'SLD_0000408.vsi - 20x_01',
 'SLD_0000409.vsi - 20x_01',
 'SLD_0000412.vsi - 20x_02',
 'SLD_0000413.vsi - 20x_02',
 'SLD_0000427.vsi - 20x_03',
 'SLD_0000427.vsi - 20x_04',
 'SLD_0000427.vsi - 20x_05',
 'SLD_0000427.vsi - 20x_06',
 'SLD_0000428.vsi - 20x_02',
 'SLD_0000429.vsi - 20x_02',
 'SLD_0000429.vsi - 20x_04',
 'SLD_0000430.vsi - 20x_02',
 'SLD_0000430.vsi - 20x_03',
 'SLD_0000431.vsi - 20x_01',
 'SLD_0000431.vsi - 20x_03',
 'SLD_0000432.vsi - 20x_03',
 'SLD_0000434.vsi - 20x_01',
 'SLD_0000434.vsi - 20x_02',
 'SLD_0000434.vsi - 20x_03',
 'SLD_0000435.vsi - 20x_04',
 'SLD_0000436.vsi - 20x_01',
 'SLD_0000436.vsi - 20x_03',
 'SLD_0000437.vsi - 20x_01',
 'SLD_0000438.